<a href="https://colab.research.google.com/github/Sopralapanca/CommonLitChallenge/blob/main/Correction%2BPOS%2BNER%2BPrepareDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LAVORO PER SARDE:
1. Aggiungere altri todo
2. Imparare ad essere umile
3. Tifare il Campobasso

TODO: CONTROLLARE I PARAGRAFI

1. Controllare se non ci sono whitespace dopo gli .\n Maiuscola
2. Score value spacy per identificare la similarità, potremmo usarla per paragrafi diversi.

TODO: CONTROLLARE I NOMI PROPRI
1. Si potrebbe effettuare il lower() prima di aggiungere le parole allo spellchecker sperando che ci siano i nomi propri necessari alla stesura del riassunto all'interno del prompt
2. Controllare i nomi propri facendo ner e non tenerli in considerazione per eventuali errori, ma confrontarli con eventuali nomi propri all'interno dei prompt (selezionare i soggetti che iniziano per le stesse lettere e poi calcolarcisi l'edit distance ==> wording, entro un certo range viene considerato match e si salva una feature per il numero di match con i prompt ==> content)



## Data importing and setup

In [1]:
!pip install spacy pyspellchecker symspellpy swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=10705c16b2c21a190462149815d708906d5927aa5cac7cc55a244a8cd3533475
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp310-cp310-linux_x86_64.whl size=190885 sha256=abd0c8c8d5d4f559d19b58cc04870210201ef63ca092a4edd9db1172ed452bdc
  Stored in directory: /root/.cache/pip/wheels/88/6a/a6/a1283cc145323a1fb3d47

In [2]:
!python -m spacy download en_core_web_sm

2023-09-16 09:56:34.200294: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-16 09:56:36.571286: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-16 09:56:39.360885: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-16 09:56:39.361544: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [3]:
!git clone "https://github.com/Sopralapanca/CommonLitChallenge.git"

Cloning into 'CommonLitChallenge'...
remote: Enumerating objects: 269, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 269 (delta 65), reused 70 (delta 52), pack-reused 171
Receiving objects: 100% (269/269), 19.80 MiB | 14.15 MiB/s, done.
Resolving deltas: 100% (138/138), done.


In [4]:
import pandas as pd
import swifter

summaries_train_path = "/content/CommonLitChallenge/data/summaries_train.csv"
prompt_train_path = "/content/CommonLitChallenge/data/prompts_train.csv"

train_data = pd.read_csv(summaries_train_path, sep=',', index_col=0)
prompt_data = pd.read_csv(prompt_train_path, sep=',', index_col=0)

## Language Processing

> Definition of the Glossary \\
> Creation of the word list \\
> Language Model Class construction

In [5]:
# @title Glossary def.
GLOSSARY = {
    "ADJ": "adjective",
    "ADP": "adposition",
    "ADV": "adverb",
    "AUX": "auxiliary",
    "CONJ": "conjunction",
    "CCONJ": "coordinating conjunction",
    "DET": "determiner",
    "INTJ": "interjection",
    "NOUN": "noun",
    "NUM": "numeral",
    "PART": "particle",
    "PRON": "pronoun",
    "PROPN": "proper noun",
    "PUNCT": "punctuation",
    "SCONJ": "subordinating conjunction",
    "SYM": "symbol",
    "VERB": "verb",
    "X": "other",
    "EOL": "end of line",
    "SPACE": "space",
    ".": "punctuation mark, sentence closer",
    ",": "punctuation mark, comma",
    "-LRB-": "left round bracket",
    "-RRB-": "right round bracket",
    "``": "opening quotation mark",
    '""': "closing quotation mark",
    "''": "closing quotation mark",
    ":": "punctuation mark, colon or ellipsis",
    "$": "symbol, currency",
    "#": "symbol, number sign",
    "AFX": "affix",
    "CC": "conjunction, coordinating",
    "CD": "cardinal number",
    "DT": "determiner",
    "EX": "existential there",
    "FW": "foreign word",
    "HYPH": "punctuation mark, hyphen",
    "IN": "conjunction, subordinating or preposition",
    "JJ": "adjective (English), other noun-modifier (Chinese)",
    "JJR": "adjective, comparative",
    "JJS": "adjective, superlative",
    "LS": "list item marker",
    "MD": "verb, modal auxiliary",
    "NIL": "missing tag",
    "NN": "noun, singular or mass",
    "NNP": "noun, proper singular",
    "NNPS": "noun, proper plural",
    "NNS": "noun, plural",
    "PDT": "predeterminer",
    "POS": "possessive ending",
    "PRP": "pronoun, personal",
    "PRP$": "pronoun, possessive",
    "RB": "adverb",
    "RBR": "adverb, comparative",
    "RBS": "adverb, superlative",
    "RP": "adverb, particle",
    "TO": 'infinitival "to"',
    "UH": "interjection",
    "VB": "verb, base form",
    "VBD": "verb, past tense",
    "VBG": "verb, gerund or present participle",
    "VBN": "verb, past participle",
    "VBP": "verb, non-3rd person singular present",
    "VBZ": "verb, 3rd person singular present",
    "WDT": "wh-determiner",
    "WP": "wh-pronoun, personal",
    "WP$": "wh-pronoun, possessive",
    "WRB": "wh-adverb",
    "SP": "space (English), sentence-final particle (Chinese)",
    "ADD": "email",
    "NFP": "superfluous punctuation",
    "GW": "additional word in multi-word expression",
    "XX": "unknown",
    "BES": 'auxiliary "be"',
    "HVS": 'forms of "have"',
    "_SP": "whitespace",
    "$(": "other sentence-internal punctuation mark",
    "$,": "comma",
    "$.": "sentence-final punctuation mark",
    "ADJA": "adjective, attributive",
    "ADJD": "adjective, adverbial or predicative",
    "APPO": "postposition",
    "APPR": "preposition; circumposition left",
    "APPRART": "preposition with article",
    "APZR": "circumposition right",
    "ART": "definite or indefinite article",
    "CARD": "cardinal number",
    "FM": "foreign language material",
    "ITJ": "interjection",
    "KOKOM": "comparative conjunction",
    "KON": "coordinate conjunction",
    "KOUI": 'subordinate conjunction with "zu" and infinitive',
    "KOUS": "subordinate conjunction with sentence",
    "NE": "proper noun",
    "NNE": "proper noun",
    "PAV": "pronominal adverb",
    "PROAV": "pronominal adverb",
    "PDAT": "attributive demonstrative pronoun",
    "PDS": "substituting demonstrative pronoun",
    "PIAT": "attributive indefinite pronoun without determiner",
    "PIDAT": "attributive indefinite pronoun with determiner",
    "PIS": "substituting indefinite pronoun",
    "PPER": "non-reflexive personal pronoun",
    "PPOSAT": "attributive possessive pronoun",
    "PPOSS": "substituting possessive pronoun",
    "PRELAT": "attributive relative pronoun",
    "PRELS": "substituting relative pronoun",
    "PRF": "reflexive personal pronoun",
    "PTKA": "particle with adjective or adverb",
    "PTKANT": "answer particle",
    "PTKNEG": "negative particle",
    "PTKVZ": "separable verbal particle",
    "PTKZU": '"zu" before infinitive',
    "PWAT": "attributive interrogative pronoun",
    "PWAV": "adverbial interrogative or relative pronoun",
    "PWS": "substituting interrogative pronoun",
    "TRUNC": "word remnant",
    "VAFIN": "finite verb, auxiliary",
    "VAIMP": "imperative, auxiliary",
    "VAINF": "infinitive, auxiliary",
    "VAPP": "perfect participle, auxiliary",
    "VMFIN": "finite verb, modal",
    "VMINF": "infinitive, modal",
    "VMPP": "perfect participle, modal",
    "VVFIN": "finite verb, full",
    "VVIMP": "imperative, full",
    "VVINF": "infinitive, full",
    "VVIZU": 'infinitive with "zu", full',
    "VVPP": "perfect participle, full",
    "XY": "non-word containing non-letter",
    "IJ": "interjection",
    "NR": "proper noun",
    "NT": "temporal noun",
    "OD": "ordinal number",
    "ON": "onomatopoeia",
    "PN": "pronoun",
    "PU": "punctuation",
    "NP": "noun phrase",
    "PP": "prepositional phrase",
    "VP": "verb phrase",
    "ADVP": "adverb phrase",
    "ADJP": "adjective phrase",
    "SBAR": "subordinating conjunction",
    "PRT": "particle",
    "PNP": "prepositional noun phrase",
    "acl": "clausal modifier of noun (adjectival clause)",
    "acomp": "adjectival complement",
    "advcl": "adverbial clause modifier",
    "advmod": "adverbial modifier",
    "agent": "agent",
    "amod": "adjectival modifier",
    "appos": "appositional modifier",
    "attr": "attribute",
    "aux": "auxiliary",
    "auxpass": "auxiliary (passive)",
    "case": "case marking",
    "cc": "coordinating conjunction",
    "ccomp": "clausal complement",
    "clf": "classifier",
    "complm": "complementizer",
    "compound": "compound",
    "conj": "conjunct",
    "cop": "copula",
    "csubj": "clausal subject",
    "csubjpass": "clausal subject (passive)",
    "dative": "dative",
    "dep": "unclassified dependent",
    "det": "determiner",
    "discourse": "discourse element",
    "dislocated": "dislocated elements",
    "dobj": "direct object",
    "expl": "expletive",
    "fixed": "fixed multiword expression",
    "flat": "flat multiword expression",
    "goeswith": "goes with",
    "hmod": "modifier in hyphenation",
    "hyph": "hyphen",
    "infmod": "infinitival modifier",
    "intj": "interjection",
    "iobj": "indirect object",
    "list": "list",
    "mark": "marker",
    "meta": "meta modifier",
    "neg": "negation modifier",
    "nmod": "modifier of nominal",
    "nn": "noun compound modifier",
    "npadvmod": "noun phrase as adverbial modifier",
    "nsubj": "nominal subject",
    "nsubjpass": "nominal subject (passive)",
    "nounmod": "modifier of nominal",
    "npmod": "noun phrase as adverbial modifier",
    "num": "number modifier",
    "number": "number compound modifier",
    "nummod": "numeric modifier",
    "oprd": "object predicate",
    "obj": "object",
    "obl": "oblique nominal",
    "orphan": "orphan",
    "parataxis": "parataxis",
    "partmod": "participal modifier",
    "pcomp": "complement of preposition",
    "pobj": "object of preposition",
    "poss": "possession modifier",
    "possessive": "possessive modifier",
    "preconj": "pre-correlative conjunction",
    "prep": "prepositional modifier",
    "prt": "particle",
    "punct": "punctuation",
    "quantmod": "modifier of quantifier",
    "rcmod": "relative clause modifier",
    "relcl": "relative clause modifier",
    "reparandum": "overridden disfluency",
    "root": "root",
    "ROOT": "root",
    "vocative": "vocative",
    "xcomp": "open clausal complement",
    "ac": "adpositional case marker",
    "adc": "adjective component",
    "ag": "genitive attribute",
    "ams": "measure argument of adjective",
    "app": "apposition",
    "avc": "adverbial phrase component",
    "cd": "coordinating conjunction",
    "cj": "conjunct",
    "cm": "comparative conjunction",
    "cp": "complementizer",
    "cvc": "collocational verb construction",
    "da": "dative",
    "dh": "discourse-level head",
    "dm": "discourse marker",
    "ep": "expletive es",
    "hd": "head",
    "ju": "junctor",
    "mnr": "postnominal modifier",
    "mo": "modifier",
    "ng": "negation",
    "nk": "noun kernel element",
    "nmc": "numerical component",
    "oa": "accusative object",
    "oc": "clausal object",
    "og": "genitive object",
    "op": "prepositional object",
    "par": "parenthetical element",
    "pd": "predicate",
    "pg": "phrasal genitive",
    "ph": "placeholder",
    "pm": "morphological particle",
    "pnc": "proper noun component",
    "rc": "relative clause",
    "re": "repeated element",
    "rs": "reported speech",
    "sb": "subject",
    "sbp": "passivized subject (PP)",
    "sp": "subject or predicate",
    "svp": "separable verb prefix",
    "uc": "unit component",
    "vo": "vocative",
    "PERSON": "People, including fictional",
    "NORP": "Nationalities or religious or political groups",
    "FACILITY": "Buildings, airports, highways, bridges, etc.",
    "FAC": "Buildings, airports, highways, bridges, etc.",
    "ORG": "Companies, agencies, institutions, etc.",
    "GPE": "Countries, cities, states",
    "LOC": "Non-GPE locations, mountain ranges, bodies of water",
    "PRODUCT": "Objects, vehicles, foods, etc. (not services)",
    "EVENT": "Named hurricanes, battles, wars, sports events, etc.",
    "WORK_OF_ART": "Titles of books, songs, etc.",
    "LAW": "Named documents made into laws.",
    "LANGUAGE": "Any named language",
    "DATE": "Absolute or relative dates or periods",
    "TIME": "Times smaller than a day",
    "PERCENT": 'Percentage, including "%"',
    "MONEY": "Monetary values, including unit",
    "QUANTITY": "Measurements, as of weight or distance",
    "ORDINAL": '"first", "second", etc.',
    "CARDINAL": "Numerals that do not fall under another type",
    "PER": "Named person or family.",
    "MISC": "Miscellaneous entities, e.g. events, nationalities, products or works of art",
    "EVT": "Festivals, cultural events, sports events, weather phenomena, wars, etc.",
    "PROD": "Product, i.e. artificially produced entities including speeches, radio shows, programming languages, contracts, laws and ideas",
    "DRV": "Words (and phrases?) that are dervied from a name, but not a name in themselves, e.g. 'Oslo-mannen' ('the man from Oslo')",
    "GPE_LOC": "Geo-political entity, with a locative sense, e.g. 'John lives in Spain'",
    "GPE_ORG": "Geo-political entity, with an organisation sense, e.g. 'Spain declined to meet with Belgium'",
}

### Creating the list of words

In [6]:
import itertools
word_list = []
for prompt, query, title in zip(prompt_data.prompt_text.tolist(), prompt_data.prompt_question.tolist(), prompt_data.prompt_title.tolist()):
  word_list.append(prompt.replace('\n', ' ').split())
  word_list.append(query.replace('\n', ' ').split())
  word_list.append(title.replace('\n', ' ').split())
token_list = list(itertools.chain(*word_list))

### Language Model from Spacy


In [7]:
import spacy
import numpy as np
import pkg_resources
from symspellpy import SymSpell

class LanguageMagician():
  def __init__(self) -> None:
    # Initialization of Spacy
    self.nlp = spacy.load("en_core_web_sm")
    self.gamma = 3e-3
    self.modulus = 541

    # Initialization of SymSpell
    self.sym_spell = SymSpell(max_dictionary_edit_distance=4, prefix_length=7)
    dictionary_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_dictionary_en_82_765.txt"
    )
    bigram_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_bigramdictionary_en_243_342.txt"
    )
    # term_index is the column of the term and count_index is the
    # column of the term frequency
    self.sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    self.sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

  def init_vocabulary_(self, words):
    vector_data = {word: np.random.uniform(-1, 1, (300,)) for word in words}
    vocab = self.nlp.vocab
    for word, vector in vector_data.items():
        vocab.set_vector(word, vector)

  def init_sym_vocabulary(self, words):
    #open text file
    text_file = open("vocabulary.txt", "a")
    #write string to file
    for s in words:
      text_file.write(s)

    #close file
    text_file.close()

    corpus_path = 'vocabulary.txt'
    try:
      result = self.sym_spell.create_dictionary(corpus_path)
    except:
      raise(f'error the dictionary creation has returned {result}')


  def correct(self, row):
    # max edit distance per lookup (per single word, not per whole input string)
    suggestions = self.sym_spell.lookup_compound(row, max_edit_distance=3,
                                                #  transfer_casing=True
                                                 )
    # display suggestion term, edit distance, and term frequency
    return suggestions[0].term

  def pos_scorer(self, row):
    pos_vector = dict.fromkeys(GLOSSARY.keys(), 0)
    doc = self.nlp(row)
    for d in doc:
      pos_vector[d.pos_] += 1
    single_pos_score = 0
    np_pos = np.asarray(list(pos_vector.values()))
    for i, pos in enumerate(np_pos):
      single_pos_score += int(pos)**(self.gamma*i)
    return single_pos_score % self.modulus

  def entity_map(self, row):
    doc = self.nlp(row)
    ents = [(e.text, e.label_) for e in doc.ents]
    keys = [e.label_ for e in doc.ents]
    map = {key:[] for key in keys}
    for ent in ents:
      map[ent[1]].append(ent[0])
    return map

  def entity_scorer(self, row):
    entity_vector = dict.fromkeys(GLOSSARY.keys(), 0)
    for k, v in row.items():
      entity_vector[k] = len(v)
    single_ent_score = 0
    np_ent = np.asarray(list(entity_vector.values()))
    for i, ent in enumerate(np_ent):
      single_ent_score += int(ent)**(self.gamma*i)
    return single_ent_score % self.modulus

  def semantic_similarity(self, text1, text2):
    embedding1 = self.nlp(text1)
    embedding2 = self.nlp(text2)
    return embedding1.similarity(embedding2)



lang_wizard = LanguageMagician()
lang_wizard.init_vocabulary_(token_list)
lang_wizard.init_sym_vocabulary(token_list)

In [8]:
train_data["corrected_text"] = train_data['text'].swifter.apply(lambda x: lang_wizard.correct(x))

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

## Text processing

In [9]:
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocessText(text, removal=True):
    # replace newline with space
    text = text.replace("\n", " ")
    text = text.replace('\r', ' ')

    # Normalize spaces around punctuation marks
    text = re.sub(r"[^A-Za-z0-9']", r' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Replace curly apostrophe with straight single quote
    text = text.replace('’', "'")

    # Normalize spaces around punctuation marks
    text = text.strip()

    if removal:
      # lower case
      text = text.lower()

      # split text
      words = text.split()

      # stop word removal
      words = [w for w in words if not w in stop_words]

      # stemming
      # words = [stemmer.stem(w) for w in words]

      # lemmatization
      words = [lemmatizer.lemmatize(w) for w in words]

      text = ' '.join(words)

    return text

# Preprocess the text
prompt_data["prompt_text_preprocessed"] =   prompt_data["prompt_text"].swifter.apply(lambda x: preprocessText(x))
train_data["corrected_text_preprocessed"] = train_data["corrected_text"].swifter.apply(lambda x: preprocessText(x))

train_data["text_pre_withstop"] =           train_data["text"].swifter.apply(lambda x: preprocessText(x, False))
train_data["corrected_text_pre_withstop"] = train_data["corrected_text"].swifter.apply(lambda x: preprocessText(x, False))
# Count the length of text
prompt_data["prompt_text_length"] = prompt_data["prompt_text"].swifter.apply(len)
train_data["text_length"] =         train_data["corrected_text"].swifter.apply(len)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

### N-grams finding

In [10]:
from nltk.util import ngrams
from collections import Counter
import nltk
nltk.download('punkt')  # Download the required resources for tokenization

def count_ngrams(text, n):
  words = nltk.word_tokenize(text)
  ngram_counts = Counter(ngrams(words, n))
  return ngram_counts

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
del_columns = []

for n in range(2,5):
  col = f"{n}grams-prompttext-count"
  prompt_data[col] = prompt_data.swifter.apply(lambda row: count_ngrams(row["prompt_text"], n), axis=1)
  del_columns.append(col)

  col = f"{n}grams-correctedtext-count"
  train_data[col] = train_data.swifter.apply(lambda row: count_ngrams(row["corrected_text"], n), axis=1)
  del_columns.append(col)

  col = f"{n}grams-text-count"
  train_data[col] = train_data.swifter.apply(lambda row: count_ngrams(row["text"], n), axis=1)
  del_columns.append(col)

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

### POS counter

In [12]:
# import numpy as np
# def karp_scoring(df, col):
#   def karp_pos(row):
#     pos_vector = dict.fromkeys(GLOSSARY.keys(), 0)
#     doc = nlp(row)
#     for d in doc:
#       pos_vector[d.pos_] += 1
#     modulus = sympy.randprime(len(row)-10, len(row)+10)
#     gamma = 3e-2
#     single_pos_score = 0
#     np_pos = np.asarray(list(pos_vector.values()))
#     for i, pos in enumerate(np_pos):
#       single_pos_score += int(pos)**(gamma*i)
#     return single_pos_score % modulus

train_data['text_pos'] = train_data['corrected_text'].swifter.apply(lambda row: lang_wizard.pos_scorer(row))

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

### Entity Recognition

In [13]:
# Function for matching same entities from prompt and summary
# def entity_map(row):
#   doc = nlp(row)
#   ents = [(e.text, e.label_) for e in doc.ents]
#   keys = [e.label_ for e in doc.ents]
#   map = {key:[] for key in keys}
#   for ent in ents:
#     map[ent[1]].append(ent[0])
#   return map

prompt_data['prompt_entities'] = prompt_data['prompt_text'].swifter.apply(lambda row: lang_wizard.entity_map(row))
train_data['summary_entities'] = train_data['corrected_text'].swifter.apply(lambda row: lang_wizard.entity_map(row))

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

### Different Words counter

In [14]:
def different_word_counter(row):
  row = preprocessText(row)
  words_list = []
  for w in row.split(' '):
    if w not in words_list:
      words_list.append(w)

  return len(words_list)


In [15]:
prompt_data["prompt_text_word_counter"] = prompt_data['prompt_text'].swifter.apply(lambda row: different_word_counter(row))
train_data['summary_word_counter'] = train_data['corrected_text'].swifter.apply(lambda row: different_word_counter(row))

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

In [16]:
# @title Merging the dataset
training_data = train_data.merge(prompt_data, on='prompt_id')

### Different words counter ratio

In [17]:
def word_counter_ratio(summary_word_counter, prompt_word_counter):
  return summary_word_counter/prompt_word_counter

In [18]:
training_data['word_counter_ratio'] = training_data.swifter.apply(lambda row: word_counter_ratio(row["summary_word_counter"], row["prompt_text_word_counter"]), axis=1)

### Entity Comparison

In [19]:
def dict_compare(d1, d2):
    d1_keys = set(d1.keys())
    d2_keys = set(d2.keys())
    shared_keys = d1_keys.intersection(d2_keys)
    same = 0
    for key in shared_keys:
      for val1 in d1[key]:
        for val2 in d2[key]:
          if val1 == val2:
            same += 1
    return same

training_data['matching_entities'] = training_data.swifter.apply(lambda row: dict_compare(row.prompt_entities, row.summary_entities), axis=1)

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

### N-grams counting

In [20]:
def count_cooccurring_ngrams(text, prompt_text):
    cooccurring_count = sum((text & prompt_text).values())
    return cooccurring_count

In [21]:
for n in range(2, 5):

  text_col = f"{n}grams-text-count"
  corre_col = f"{n}grams-correctedtext-count"
  prompt_col = f"{n}grams-prompttext-count"

  new_col=f"{n}grams-correct-count"
  training_data[new_col] = training_data.swifter.apply(lambda row: count_cooccurring_ngrams(row[text_col], row[corre_col]), axis=1)

  new_col=f"{n}grams-cooccurrence-count"
  training_data[new_col] = training_data.swifter.apply(lambda row: count_cooccurring_ngrams(row[corre_col], row[prompt_col]), axis=1)

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

### Semantic Similarity

In [22]:
training_data["semantic_similarity"] = training_data.swifter.apply(lambda row: lang_wizard.semantic_similarity(row["corrected_text_preprocessed"], row["prompt_text_preprocessed"]), axis=1)

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

### Misspelling finding and counting

In [23]:
from spellchecker import SpellChecker

spell = SpellChecker()

spell.word_frequency.load_words(token_list)

def misspelled(text):
  words = text.split()
  misspelled = spell.unknown(words)
  return misspelled

In [24]:
training_data["text_misspelled_words"] = training_data["text_pre_withstop"].swifter.apply(lambda x: misspelled(x))

training_data["text_misspelled_counter"] = training_data["text_misspelled_words"].swifter.apply(len)

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

### Text length ratio

In [25]:
training_data["length_ratio"] = training_data["text_length"] / training_data["prompt_text_length"]

### TF-IDF score creation

In [26]:
def add_row(df1, df2, preprocess=False):
  row = df2.unique().tolist()[0]
  if preprocess:
    row = preprocessText(row)
  combined_data = pd.concat([pd.Series([row]),df1.loc[:]]).reset_index(drop=True) #append row on the head of the dataframe
  return combined_data

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Group by 'prompt_id' and compute TF-IDF separately for each class
tfidf_vectorizers = {}

for class_id, group in training_data.groupby('prompt_id'):
    #text_data = group['text'].swifter.apply(preprocessText)
    text_data = group['corrected_text_preprocessed']

    prompt_question_data = group['prompt_question']
    prompt_title_data = group['prompt_title']
    prompt_text_data = group['prompt_text_preprocessed']

    # Concatenate the preprocessed data for TF-IDF calculation
    combined_data = add_row(text_data, prompt_question_data, True)
    combined_data = add_row(combined_data, prompt_title_data, True)
    combined_data = add_row(combined_data, prompt_text_data)


    # Compute TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(combined_data)
    tfidf_vectorizers[class_id] = {'vectorizer': tfidf_vectorizer, 'matrix': tfidf_matrix}

In [28]:
karp_tfidf_scores = {}

# Calculate TF-IDF scores for each document
for class_id, group in training_data.groupby('prompt_id'):
    tfidf_vectorizer = tfidf_vectorizers[class_id]['vectorizer']
    tfidf_matrix = tfidf_vectorizers[class_id]['matrix']
    tfidf_matrix = tfidf_matrix[3:] #remove first 3 rows f the matrix since they belongs to prompt_text, prompt_question, prompt_title

    modulus = 541

    # Iterate through documents and calculate TF-IDF scores
    for index, row in group.iterrows():
        doc_tfidf = tfidf_matrix[index - group.index[0]].toarray()[0]

        doc_tfidf = doc_tfidf[doc_tfidf>0]

        # Calculate the average TF-IDF score for the document
        #average_tfidf_score = sum(doc_tfidf) / len(doc_tfidf)

        #average_tfidf_scores[index] = average_tfidf_score

        gamma = 1e-2
        single_tfidf_score = sum([t**(gamma*i) for i, t in enumerate(doc_tfidf)]) % modulus

        karp_tfidf_scores[index] = single_tfidf_score

In [29]:
# Add the calculated average TF-IDF scores as a new column to the DataFrame
training_data['karp_tfidf_scores'] = [karp_tfidf_scores[index] for index in training_data.index]

## Dataset normalization

In [30]:
# normalize the data taking into consideration the prompt title

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def normalize_col(df, col):
    # Create a Min-Max Scaler
    scaler = MinMaxScaler()

    df[col] = df.groupby('prompt_id')[col].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten())

    return df

In [31]:
normalize_cols = ["text_length", "text_misspelled_counter",
                  "2grams-cooccurrence-count", "2grams-correct-count",
                  "3grams-cooccurrence-count", "3grams-correct-count",
                  "4grams-cooccurrence-count", "4grams-correct-count",
                  "karp_tfidf_scores"]
for col in normalize_cols:
  training_data = normalize_col(training_data, col)

In [32]:
normalize_cols.extend(['prompt_text_length'])
del_columns.extend(normalize_cols)

training_data.drop(columns=del_columns, axis=1, inplace=True)

In [33]:
training_data.head()

,prompt_id,text,content,wording,corrected_text,corrected_text_preprocessed,text_pre_withstop,corrected_text_pre_withstop,text_pos,summary_entities,...,prompt_title,prompt_text,prompt_text_preprocessed,prompt_entities,prompt_text_word_counter,word_counter_ratio,matching_entities,semantic_similarity,text_misspelled_words,length_ratio
0,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,the third wave was an experiment to see how pe...,third wave experiment see people reacted new o...,The third wave was an experimentto see how peo...,the third wave was an experiment to see how pe...,12.402646,"{'ORDINAL': ['third'], 'CARDINAL': ['one']}",...,The Third Wave,Background \nThe Third Wave experiment took pl...,background third wave experiment took place cu...,"{'ORDINAL': ['Third', 'third', 'second'], 'ORG...",205,0.146341,1,0.264169,"{experimentto, experement}",0.096910
1,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,the third wave developed rapidly because the s...,third wave developed rapidly student genuinely...,The Third Wave developed rapidly because the s...,the third wave developed rapidly because the s...,13.702154,"{'ORDINAL': ['third', 'third', 'fifth', 'third...",...,The Third Wave,Background \nThe Third Wave experiment took pl...,background third wave experiment took place cu...,"{'ORDINAL': ['Third', 'third', 'second'], 'ORG...",205,0.390244,4,0.568199,"{classparticipation, consiquenses, anounce, ge...",0.335955
2,814d6b,The third wave only started as an experiment w...,0.205683,0.380538,the third wave only started as an experiment w...,third wave started experiment within class slo...,The third wave only started as an experiment w...,the third wave only started as an experiment w...,12.392376,"{'ORDINAL': ['third'], 'PERSON': ['jones']}",...,The Third Wave,Background \nThe Third Wave experiment took pl...,background third wave experiment took place cu...,"{'ORDINAL': ['Third', 'third', 'second'], 'ORG...",205,0.131707,1,0.492028,"{partipating, mr}",0.095506
3,814d6b,The experimen was orginally about how even whe...,0.567975,0.969062,the experiment was originally about how even w...,experiment originally even terrible thing happ...,The experimen was orginally about how even whe...,the experiment was originally about how even w...,11.399366,"{'ORDINAL': ['third', 'third']}",...,The Third Wave,Background \nThe Third Wave experiment took pl...,background third wave experiment took place cu...,"{'ORDINAL': ['Third', 'third', 'second'], 'ORG...",205,0.146341,2,0.466138,"{thrid, orginally, thngs, experimen}",0.124719
4,814d6b,The third wave developed so quickly due to the...,-0.910596,-0.081769,the third wave developed so quickly due to the...,third wave developed quickly due student parti...,The third wave developed so quickly due to the...,the third wave developed so quickly due to the...,8.157947,{'ORDINAL': ['third']},...,The Third Wave,Background \nThe Third Wave experiment took pl...,background third wave experiment took place cu...,"{'ORDINAL': ['Third', 'third', 'second'], 'ORG...",205,0.063415,1,0.196725,"{particip, contro}",0.040449


In [34]:
training_data.corrwith(training_data["content"])

content                     1.000000
wording                     0.751380
text_pos                    0.581775
summary_word_counter        0.791634
prompt_text_word_counter   -0.027778
word_counter_ratio          0.802038
matching_entities           0.275606
semantic_similarity         0.450118
length_ratio                0.777919
dtype: float64

In [35]:
training_data.corrwith(training_data["wording"])

content                     0.751380
wording                     1.000000
text_pos                    0.451932
summary_word_counter        0.516272
prompt_text_word_counter   -0.134943
word_counter_ratio          0.560617
matching_entities           0.209562
semantic_similarity         0.164889
length_ratio                0.548927
dtype: float64

In [36]:
training_data.to_csv('dataset.csv', index=False)

In [37]:
!zip -r 'dataset.zip' 'dataset.csv' # download and save in github

  adding: dataset.csv (deflated 96%)
